Candidate Name: Yagna Suthar
Email: yagna.suthar@gmail.com
Mobile Number: +91 9265679968

# 1.Importing important libraries /Apis

In [282]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Importing the Both datasets

In [283]:
trades_dataset = pd.read_csv('historical_data.csv')
# print(trades_dataset.head())
sentiment_dataset = pd.read_csv('fear_greed_index.csv')
# print(sentiment_dataset.head())

trades_dataset["Timestamp IST"] = pd.to_datetime(trades_dataset["Timestamp IST"],dayfirst=True)
trades_dataset["date"] = trades_dataset["Timestamp IST"].dt.date
sentiment_dataset["date"] = pd.to_datetime(sentiment_dataset["date"]).dt.date
print("\n\n\n")
print(trades_dataset.columns)






Index(['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side',
       'Timestamp IST', 'Start Position', 'Direction', 'Closed PnL',
       'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID',
       'Timestamp', 'date'],
      dtype='object')


#2.Data Cleaning / Data Preprocessing

In [284]:
print(sentiment_dataset['classification'].value_counts())
sentiment_dataset['classification'].unique()


classification
Fear             781
Greed            633
Extreme Fear     508
Neutral          396
Extreme Greed    326
Name: count, dtype: int64


array(['Fear', 'Extreme Fear', 'Neutral', 'Greed', 'Extreme Greed'],
      dtype=object)

## Removing unnecessary columns from the dataset


In [285]:

sentiment_dataset = sentiment_dataset.iloc[:,2:]
sentiment_cleaned = sentiment_dataset[['date','classification']] # changing column order date as first column

                                      ##### Normalizing sentiment labels ######
sentiment_cleaned['sentiment'] = sentiment_cleaned['classification'].replace({
    'Fear':'Fear',
    'Extreme Fear':'Fear',
    'Greed':'Greed',
    'Extreme Greed':'Greed'
})

sentiment_cleaned = sentiment_cleaned.drop(['classification'],axis=1)

trades_dataset  = trades_dataset.drop(['Timestamp','Order ID','Transaction Hash','Crossed','Start Position','Direction','Trade ID'],axis=1)
print(sentiment_cleaned['sentiment'].value_counts())

sentiment
Fear       1289
Greed       959
Neutral     396
Name: count, dtype: int64


/tmp/ipython-input-407981110.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentiment_cleaned['sentiment'] = sentiment_cleaned['classification'].replace({


## Analyzing dataset : if there any missing or null value is present or not

In [286]:
print(sentiment_cleaned.describe(),'\n\n')
sentiment_cleaned.info()

print(trades_dataset.describe(),'\n\n')
print(trades_dataset.info())

              date sentiment
count         2644      2644
unique        2644         3
top     2025-05-02      Fear
freq             1      1289 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2644 entries, 0 to 2643
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       2644 non-null   object
 1   sentiment  2644 non-null   object
dtypes: object(2)
memory usage: 41.4+ KB
       Execution Price   Size Tokens      Size USD  \
count    211224.000000  2.112240e+05  2.112240e+05   
mean      11414.723350  4.623365e+03  5.639451e+03   
min           0.000005  8.740000e-07  0.000000e+00   
25%           4.854700  2.940000e+00  1.937900e+02   
50%          18.280000  3.200000e+01  5.970450e+02   
75%         101.580000  1.879025e+02  2.058960e+03   
max      109004.000000  1.582244e+07  3.921431e+06   
std       29447.654868  1.042729e+05  3.657514e+04   

                       Timestamp IST     Closed PnL           

## Merging the Dataset into single dataset based in date (common column)

In [287]:
final_df = trades_dataset.merge(
    sentiment_cleaned,
    on="date",
    how="left"
)
# print(final_df.info(),'\n\n\n\n')
print(final_df.head())

                                      Account  Coin  Execution Price  \
0  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9769   
1  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9800   
2  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9855   
3  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9874   
4  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9894   

   Size Tokens  Size USD Side       Timestamp IST  Closed PnL       Fee  \
0       986.87   7872.16  BUY 2024-12-02 22:50:00         0.0  0.345404   
1        16.00    127.68  BUY 2024-12-02 22:50:00         0.0  0.005600   
2       144.09   1150.63  BUY 2024-12-02 22:50:00         0.0  0.050431   
3       142.98   1142.04  BUY 2024-12-02 22:50:00         0.0  0.050043   
4         8.73     69.75  BUY 2024-12-02 22:50:00         0.0  0.003055   

         date sentiment  
0  2024-12-02     Greed  
1  2024-12-02     Greed  
2  2024-12-02     Greed  
3  2024-12-0

In [288]:
print(final_df['sentiment'].value_counts())
print(final_df.isnull().sum(),'\n\n\n')

# if there are very less missing values compare to main dataset(214k samples) then we drop those samples
final_df = final_df.dropna(subset=['sentiment'])
print(final_df.isnull().sum())

sentiment
Greed      90295
Fear       83237
Neutral    37686
Name: count, dtype: int64
Account            0
Coin               0
Execution Price    0
Size Tokens        0
Size USD           0
Side               0
Timestamp IST      0
Closed PnL         0
Fee                0
date               0
sentiment          6
dtype: int64 



Account            0
Coin               0
Execution Price    0
Size Tokens        0
Size USD           0
Side               0
Timestamp IST      0
Closed PnL         0
Fee                0
date               0
sentiment          0
dtype: int64


#3.Saving final merged csv file to desired location

In [289]:
# Make sure the folder exists

import os

if not os.path.exists('csv_files'):
    os.makedirs('csv_files')

# Save the final dataframe as CSV inside csv_files
final_df.to_csv('csv_files/final_df.csv', index=False)

print("final_df.csv saved successfully in csv_files/")

final_df.csv saved successfully in csv_files/
